In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from typing import Callable
import pandas as pd

In [2]:
a = np.array([[1, 0, 2], [2, 2, 1]])
a.cumsum(axis=1)

a > 0

array([[ True, False,  True],
       [ True,  True,  True]])

In [3]:
def SPRT(a, b, mu) -> (float, float, float, float):
    A = b / (1 - a)
    B = (1 - b) / a
    
    #First order
    X = ss.norm.rvs(loc = 0, scale = 1, size=(10000, 1000))
    Y = ss.norm.logpdf(X, loc = 0, scale = 1) - ss.norm.logpdf(X, loc = mu, scale = 1)
    Z = Y.cumsum(axis=1)
    T = -(np.log(A) <= Z).astype(int) * (np.log(B) >= Z).astype(int)
    E_0 = T.mean()
    alp = np.sum(T > np.log(B))
    
    #Second order
    X = ss.norm.rvs(loc = mu, scale = 1, size=(10000, 1000))
    Y = ss.norm.logpdf(X, loc = 0, scale = 1) - ss.norm.logpdf(X, loc = mu, scale = 1)
    Z = Y.cumsum(axis=1)
    
    T0 = np.min(np.where(np.log(A) > Z), axis = 1)
    T1 = np.min(np.where(np.log(B) < Z), axis = 1)
    T = min(T0, T1)
    E_1 = T.mean()
    bet = np.sum(T < np.log(A))
    
    return alp, bet, E_0, E_1
    

#  Домашка.
## 1.

In [4]:
df = pd.read_csv('hw_6_data.txt', sep=' ', names=['first', 'second', 'third', 'winning', 'reporter?'])
df.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [5]:
report = df[df['reporter?'] == 1]
report.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [6]:
report = report - (report > 4)
report.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,4,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [7]:
expected = np.array([1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 /64, 1 / 64, 1 / 32])

names = ['first', 'second', 'third']
observed = np.unique(report[names], return_counts=True)[1]
observed

ss.chisquare(observed, expected * 138 * 3)

Power_divergenceResult(statistic=6.178743961352657, pvalue=0.4034680378126573)

## 2.


In [8]:
f = ss.expon().rvs(size=20, random_state=1337)

In [45]:
def likehood(a, f):
    return np.log(a) + np.mean(f) / a

In [12]:
from scipy.optimize import minimize

In [46]:
ret = minimize(likehood, 1, args = f)
ret

      fun: 0.9655351651813455
 hess_inv: array([[1]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 1
     njev: 2
   status: 0
  success: True
        x: array([0.96612231])

In [47]:
theta = ret.x[0]
theta

0.9661223068833351

In [48]:
stat = ss.ks_1samp(f, ss.expon(scale = theta).cdf).statistic
stat

0.12914935983337905

In [ ]:
BIG = 50000
gf = ss.expon().rvs(size=(BIG, 20), random_state=1337)

In [53]:
sum(np.apply_along_axis(lambda a: ss.ks_1samp(a,
                                                 ss.expon(scale = minimize(likehood, 1, args = a).x[0]).cdf).statistic,
                           1, gf) > stat) / BIG

0.70944

## 3.


In [54]:
np.random.seed(1337)
a = np.zeros(100, dtype=int)
for _ in range(500):
    a[np.random.randint(100)] += 1

In [55]:
np.unique(a, return_counts = True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([ 3, 10, 14, 18, 18, 15,  7,  6,  6,  1,  2], dtype=int64))

In [100]:
bounds = [2, 3, 4, 5, 6, 8, 11]
left = 0
observed = []
for right in bounds:
    observed.append(sum([left < t <= right for t in a]) / 100)
    left = right
observed = np.array(observed)
observed

array([0.13, 0.14, 0.18, 0.18, 0.15, 0.13, 0.09])

In [101]:
def chi(l, obs):
    bounds = [2, 3, 4, 5, 6, 8, 11]
    exp = []
    left= 0
    for right in bounds:
        exp.append(ss.poisson.cdf(right, mu=l) - ss.poisson.cdf(left, mu=l))
        left = right
    exp = np.array(exp) * 100
    return -ss.chisquare(obs, exp)[0][0]

In [102]:
ret = minimize(chi, 2, args = observed)
ret

      fun: -96.97729356361752
 hess_inv: array([[0.60771489]])
      jac: array([2.86102295e-06])
  message: 'Optimization terminated successfully.'
     nfev: 14
      nit: 6
     njev: 7
   status: 0
  success: True
        x: array([4.90798528])